# Generation.ipynb


## Note: THIS IS NOT THE MOST UP-TO-DATE SCRIPT
    To see the most current version, look to "/media/data/jacob/GitHub/genetic_algorithm/Notebooks/generation.ipynb"
This is a backup version of the main notebook "generation.ipynb," created at ~1030:pm 11-28-2020 by Jacob Rose, since the the notebook was starting to get cluttered and making it difficult to think. This notebook preserves several possible design directions which were causing more complications than tolerable at this stage in the process, but may be worth returning to upon future refactorizations. These mainly include the custom class definitions for Chromosome and ChromosomeOptions, which atm need a simpler data model to work from

In [1]:
# train_ds = data['train'].map(lambda x,y: (resize(x),y)).shuffle(1024).cache().batch(config.batch_size).prefetch(-1)
def get_hardest_k_examples(test_dataset, model, k=32):
    class_probs = model.predict(test_dataset)
    predictions = np.argmax(class_probs, axis=1)
    losses = tf.keras.losses.categorical_crossentropy(test_dataset.y, class_probs)
    argsort_loss =  np.argsort(losses)

    highest_k_losses = np.array(losses)[argsort_loss[-k:]]
    hardest_k_examples = test_dataset.x[argsort_loss[-k:]]
    true_labels = np.argmax(test_dataset.y[argsort_loss[-k:]], axis=1)

    return highest_k_losses, hardest_k_examples, true_labels, predictions
        
def log_high_loss_examples(test_dataset, model, k=32):
    print(f'logging k={k} hardest examples')
    losses, hardest_k_examples, true_labels, predictions = get_hardest_k_examples(test_dataset, model, k=k)
    wandb.log(
        {"high-loss-examples":
                            [wandb.Image(hard_example, caption = f'true:{label},\npred:{pred}\nloss={loss}')
                             for hard_example, label, pred, loss in zip(hardest_k_examples, true_labels, predictions, losses)]
        })

from IPython.display import display
import warnings
warnings.filterwarnings('ignore')
from pyleaves.utils import set_tf_config
set_tf_config(num_gpus=1)

import wandb
from wandb.keras import WandbCallback
# wandb.login()

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, ReLU, ELU, LeakyReLU, Flatten, Dense, Add, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers.experimental.preprocessing import StringLookup, CategoryEncoding
import pprint
pp = pprint.PrettyPrinter(indent=4)

import matplotlib.pyplot as plt
import numpy as np
np.random.seed(666)
tf.random.set_seed(666)

from typing import List, Tuple, Union, Dict, NamedTuple
import tensorflow_datasets as tfds
from omegaconf import OmegaConf

from tfrecord_utils.img_utils import resize_repeat
from boltons.funcutils import partial

# import logging
# logger = logging.getLogger('')

LOG_DIR = '/media/data/jacob/GitHub/evolution_logs'
import os
os.makedirs(LOG_DIR, exist_ok=True)
from paleoai_data.utils.logging_utils import get_logger
logger = get_logger(logdir=LOG_DIR, filename='generation_evolution_logs.log', append=True)

!nvidia-smi


setGPU: Setting GPU to: [0]
Initial visible GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
visible GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Successfully set memory_growth=True and limited GPUs visible to tensorflow.

Now using GPU(s):
['/physical_device:GPU:0']


/media/data/conda/jacob/envs/pyleaves2.3/lib/python3.7/site-packages/wandb/vendor/graphql-core-1.1/graphql/type/directives.py:55: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  assert isinstance(locations, collections.Iterable), 'Must provide locations for directive.'


Sat Nov 28 00:58:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN X (Pascal)    Off  | 00000000:04:00.0 Off |                  N/A |
| 23%   21C    P8     7W / 250W |     10MiB / 12196MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN X (Pascal)    Off  | 00000000:05:00.0 Off |                  N/A |
| 23%   23C    P8     8W / 250W |     10MiB / 12196MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

## Data Definitions

In [2]:
TFDS_DATASETS = ['plant_village']

class ClassLabelEncoder:
    def __init__(self, ds_info: tfds.core.dataset_info.DatasetInfo):
        self.info = ds_info
        self.dataset_name = ds_info.full_name
        self.num_samples = ds_info.splits['train'].num_examples
        self.num_classes = ds_info.features['label'].num_classes
        self.class_list = ds_info.features['label'].names
        self._str2int = ds_info.features['label'].str2int
        self._int2str = ds_info.features['label'].int2str
        
    def str2int(self, labels: Union[List[str],Tuple[str]]):
        labels = _valid_eager_tensor(labels)
        if not isinstance(labels, [list, tuple]):
            assert isinstance(labels, str)
            labels = [labels]
        return [self._str2int(l) for l in labels]
    
    def int2str(self, labels: Union[List[int],Tuple[int]]):
        labels = _valid_eager_tensor(labels)
        if not isinstance(labels, [list, tuple]):
            assert isinstance(labels, (int, np.int64))
            labels = [labels]
        return [self._int2str(l) for l in labels]
    
    def one_hot(self, label: tf.int64):
        '''
        One-Hot encode integer labels
        Use tf.data.Dataset.map(lambda x,y: (x, encoder.one_hot(y))) and pass in individual labels already encoded in int64 format.
        '''
        return tf.one_hot(label, depth=self.num_classes)
    
    def __repr__(self):
        return f'''Dataset Name: {self.dataset_name}
        Num_samples: {self.num_samples}
        Num_classes: {self.num_classes}'''
    
    def _valid_eager_tensor(self, tensor, strict=False):
        '''
        If tensor IS an EagerTensor, return tensor.numpy(). 
        if strict==True, and tensor IS NOT an EagerTensor, then raise AssertionError.
        if strict==False, and tensor IS NOT an EagerTensor, then return tensor without modification 
        '''
        try:
            assert isinstance(labels, tf.python.framework.ops.EagerTensor)
        except AssertionError:
            if strict:
                raise AssertionError(f'Strict EagerTensor requirement failed assertion test in ClassLabelEncoder._valid_eager_tensor method')
        labels = labels.numpy()
        return labels

def load_plant_village_dataset(split=['train'],
                               data_dir=None,
                               batch_size=None):
    
    builder = tfds.builder('plant_village', data_dir=data_dir)
    ds_info = builder.info
    builder.download_and_prepare()

    print(f'splits: {split}')
    data = builder.as_dataset(split=list(split),
                              shuffle_files=True,
                              batch_size=batch_size,
                              as_supervised=True
                              )
    
    if not isinstance(data, (tuple, list)):
        data = {'train':data}
    elif len(data)==2:
        data = {'train':data[0], 'val':data[1]}
    elif len(data)==3:
        data = {'train':data[0], 'val':data[1], 'test':data[2]}
    
    return data, builder

def load_tfds_dataset(dataset_name='plant_village', 
                      split={'train':'train'},
                      data_dir=None,
                      batch_size=None):
    '''
    General interface function to properly route users to the correct function for loading their queried dataset from Tensorflow Datasets (TFDS) public data.
    '''
    assert dataset_name in TFDS_DATASETS
    
    print(f'Getting the TFDS dataset: {dataset_name}')
    if dataset_name == 'plant_village':
        return load_plant_village_dataset(split      =split,
                                          data_dir   =data_dir,
                                          batch_size =batch_size)
    else:
        raise Exception('Attempted to load dataset from TFDS that we have yet to build an adapter for. Consider building a minimal working prototype by using alternative datasets as a template.')
    

def get_parse_example_func(target_size, num_classes):
    resize = resize_repeat(target_size=tuple(target_size), training=False)
    one_hot = partial(tf.one_hot, depth=num_classes)
    def _parse_example(x, y):
        x = tf.image.convert_image_dtype(x, tf.float32)
        x = resize(x)
        y = one_hot(y)
        return x,y
    return _parse_example

def preprocess_data(data: tf.data.Dataset, target_size=None, num_classes=None, batch_size=1): #class_encoder=None):
    parse_example = get_parse_example_func(target_size=target_size, num_classes=num_classes) #class_encoder=class_encoder)
    return data.map(lambda x,y: parse_example(x, y)) \
                .shuffle(1024) \
                .batch(batch_size) \
                .prefetch(-1)



def load_and_preprocess_data(data_config):

    data, builder = load_tfds_dataset(dataset_name=data_config.load.dataset_name,
                                      split=data_config.load.split,
                                      data_dir=data_config.load.data_dir)

    data_info     = builder.info
    class_encoder = ClassLabelEncoder(data_info)
    print(class_encoder)
#     vocab = class_encoder.class_list
    preprocess = partial(preprocess_data,
                         batch_size=data_config.preprocess.batch_size,
                         target_size=data_config.preprocess.target_size,
                         num_classes=class_encoder.num_classes)

    data['train'] = preprocess(data=data['train']) #, batch_size=config.batch_size)
    data['val'] = preprocess(data=data['val']) #, batch_size=config.batch_size)
    data['test'] = preprocess(data=data['test']) #, batch_size=config.batch_size)
    
    return data, class_encoder

/media/data/conda/jacob/envs/pyleaves2.3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
# def load_and_log(dataset_name='plant_village', 
#                  data_dir    = '/media/data/jacob/tensorflow_datasets'):
# dataset_name='plant_village'
# data_dir = '/media/data/jacob/tensorflow_datasets'
# if True:
#     split = {
#              'train': 'train[0%:60%]',
#              'val': 'train[60%:70%]',
#              'test': 'train[70%:100%]'
#             }
#     with wandb.init(project="artifacts-example", job_type="load-data") as run:
#     run   = wandb.init(project="artifacts-example", job_type="load-data")        
#     data, builder = load_tfds_dataset(dataset_name=dataset_name,
#                                       split=split,
#                                       data_dir=data_dir,
#                                       batch_size=None)
#     data_info     = builder.info
#     raw_data      = wandb.Artifact(
#                                     f"{dataset_name}-raw", type="dataset",
                                    
#                                     description="Raw {plant_village} dataset, split into train/val/test",
#                                     metadata={"source": "keras.datasets.mnist",
#                                               "sizes": [len(dataset.x) for dataset in datasets]})
# split = {
#          'train': 'train[0%:60%]',
#          'val': 'train[60%:70%]',
#          'test': 'train[70%:100%]'
#         }    
# data_dir = '/media/data/jacob/tensorflow_datasets'
# data, ds_info = load_plant_village_dataset(split=split,
#                                          data_dir=data_dir)

## Creating and tracking label encoders

In [4]:
dataset_name='plant_village'
data_dir = '/media/data/jacob/tensorflow_datasets'

exp_config = OmegaConf.create({'seed':756, #237,
                               'batch_size':16,
                               'input_shape':(224,224,3),
                               'output_size':38,
                               'epochs_per_organism':3
                              })

data_config = OmegaConf.create({'load':{},'preprocess':{}})

data_config['load'] = {'dataset_name':'plant_village',
                       'split':['train[0%:60%]','train[60%:70%]','train[70%:100%]'],
                       'data_dir':'/media/data/jacob/tensorflow_datasets'}

data_config['preprocess'] = {'batch_size':exp_config.batch_size,
                             'target_size':exp_config.input_shape[:2]}

organism_config = OmegaConf.create({'input_shape':exp_config.input_shape,
                                    'output_size':38,
                                    'epochs_per_organism':5})
generation_config = OmegaConf.create({
                                      'population_size':5,
                                      'num_generations_per_phase':3,
                                      'fitSurvivalRate': 0.5,
                                      'unfitSurvivalProb':0.2,
                                      'mutationRate':0.1,
                                      'num_phases':5
                                    })

In [5]:
# class OrganismConfig(NamedTuple):
#     """
#     The best run found by an hyperparameter search (see :class:`~transformers.Trainer.hyperparameter_search`).
#     Parameters:
#         run_id (:obj:`str`):
#             The id of the best run (if models were saved, the corresponding checkpoint will be in the folder ending
#             with run-{run_id}).
#         objective (:obj:`float`):
#             The objective that was obtained for this run.
#         hyperparameters (:obj:`Dict[str, Any]`):
#             The hyperparameters picked to get this run.
#     """
    
    
#     input_shape: Tuple[int]
#     output_size: int
#     epochs_per_organism: int=5

#     run_id: str
#     objective: float
#     hyperparameters: Dict[str, Any]


# def default_compute_objective(metrics: Dict[str, float]) -> float:
#     """
#     The default objective to maximize/minimize when doing an hyperparameter search. It is the evaluation loss if no
#     metrics are provided to the :class:`~transformers.Trainer`, the sum of all metrics otherwise.
#     Args:
#         metrics (:obj:`Dict[str, float]`): The metrics returned by the evaluate method.
#     Return:
#         :obj:`float`: The objective to minimize or maximize
#     """
#     metrics = copy.deepcopy(metrics)
#     loss = metrics.pop("eval_loss", None)
#     _ = metrics.pop("epoch", None)
#     return loss if len(metrics) == 0 else sum(metrics.values())


# def default_hp_space_optuna(trial) -> Dict[str, float]:
#     from .integrations import is_optuna_available

#     assert is_optuna_available(), "This function needs Optuna installed: `pip install optuna`"
#     return {
#         "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
#         "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 5),
#         "seed": trial.suggest_int("seed", 1, 40),
#         "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16, 32, 64]),
#     }




# data_dir = data_config.load.data_dir
# split = data_config.load.split
# batch_size = None

# builder = tfds.builder('plant_village', data_dir=data_dir)
# ds_info = builder.info
# builder.download_and_prepare()

# # print(f'splits: {split}')

# # data = builder.as_dataset(as_supervised=True,
# #                           split=split)

# # # data = builder.as_dataset(split=split,
# # #                           shuffle_files=True,
# # #                           batch_size=batch_size,
# # #                           as_supervised=True
# # #                           )

# data

In [6]:
data, class_encoder = load_and_preprocess_data(data_config)

INFO Load dataset info from /media/data/jacob/tensorflow_datasets/plant_village/1.0.2
INFO Reusing dataset plant_village (/media/data/jacob/tensorflow_datasets/plant_village/1.0.2)
INFO Constructing tf.data.Dataset for split ['train[0%:60%]', 'train[60%:70%]', 'train[70%:100%]'], from /media/data/jacob/tensorflow_datasets/plant_village/1.0.2


Getting the TFDS dataset: plant_village
splits: ['train[0%:60%]', 'train[60%:70%]', 'train[70%:100%]']
Dataset Name: plant_village/1.0.2
        Num_samples: 54303
        Num_classes: 38


# Organism
An organism contains the following:

1. phase - This denotes which phase does the organism belong to
2. chromosome - A dictionary of genes (hyperparameters)
3. model - The `tf.keras` model corresponding to the chromosome
4. prevBestOrganism - The best organism in the previous **phase**

In [72]:
import json
import logging

class Params():
    """Class that loads hyperparameters from a json file.
    Example:
    ```
    params = Params(json_path)
    print(params.learning_rate)
    params.learning_rate = 0.5  # change the value of learning_rate in params
    ```
    """

    def __init__(self, json_path):
        self.update(json_path)

    @classmethod
    def save(self, json_path):
        """Saves parameters to json file"""
        with open(json_path, 'w') as f:
            json.dump(self.__dict__, f, indent=4)

    def update(self, json_path):
        """Loads parameters from json file"""
        with open(json_path) as f:
            params = json.load(f)
            self.__dict__.update(params)

    def update(self, json_path):
        """Loads parameters from json file"""
        with open(json_path) as f:
            params = json.load(f)
            self.__dict__.update(params)

            
    @property
    def dict(self):
        """Gives dict-like access to Params instance by `params.dict['learning_rate']`"""
        return self.__dict__

    
    
    
class Stateful(object):

    def get_state(self):
        """Returns the current state of this object.
        This method is called during `save`.
        """
        raise NotImplementedError

    def set_state(self, state):
        """Sets the current state of this object.
        This method is called during `reload`.
        # Arguments:
          state: Dict. The state to restore for this object.
        """
        raise NotImplementedError

    def save(self, fname):
        """Saves this object using `get_state`.
        # Arguments:
          fname: The file name to save to.
        """
        state = self.get_state()
        state_json = json.dumps(state)
        with tf.io.gfile.GFile(fname, 'w') as f:
            f.write(state_json)
        return str(fname)

    def reload(self, fname):
        """Reloads this object using `set_state`.
        # Arguments:
          fname: The file name to restore from.
        """
        with tf.io.gfile.GFile(fname, 'r') as f:
            state_data = f.read()
        state = json.loads(state_data)
        self.set_state(state)

/media/data/conda/jacob/envs/pyleaves2.3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
params = Params()

In [62]:
import pandas as pd
import json
from box import Box
from bunch import Bunch
from pprint import pprint as pp
import random

class BaseOptions(NamedTuple):

    __indent = None
    __hparams = 
    __chromosomes = {}
    
    @property
    def data(self):
        return self.__data
    
    @property
    def chromosomes(self):
        return self.__chromosomes
        
    def keys(self):
        return list(vars(self).keys())
    
    def values(self):
        return list(vars(self).values())

    def genes(self):
        return self.keys()
    
    def variants(self):
        return self.values()
    
    def __dumps__(self, indent=None):
        return json.dumps(self.chromosomes, indent=indent)
    
    @classmethod
    def __loads__(cls, json_string):
        loaded = cls()
        loaded.__init__(**json.loads(json_string))
        return loaded
    
    @classmethod
    def __save__(self, json_path):
        """Saves parameters to json file"""
        with open(json_path, 'w') as f:
            json.dump(self.data, f, indent=4)
    
    @property
    def serialized(self):
        '''
        Args:
            serialize (bool): default=False
                Get the JSON-compatible string representation of the ChromosomeOptions instance
        '''
        return self.__dumps__(indent=self.__indent)
    
    @classmethod
    def from_serialized(cls, serialized: str):
        return cls.__loads__(serialized)
    
    def __repr__(self):
        return '\n'.join([f'{k}:\n\t{v}' for k,v in self.chromosomes.items()])
        
    def __hash__(self):
        return hash(self.serialized)
        
    def __eq__(self, other):
        return hash(self)==hash(other)


# class Chromosome(BaseOptions):
class BaseChromosome:#(NamedTuple):
    
    def __init__(self,
                 **kwargs):
        self.__chromosome = {k:v for k,v in kwargs.items() if k not in ['seed']}
#         self.__chromosomes = {k:v for k,v in locals().items() if k!='self' and not k.startswith('__')}

    @property
    def chromosome(self):
        return self.__chromosome

    def __repr__(self):
        return '\n'.join([f'{k}:\n\t{v}' for k,v in self.chromosome.items()])
    
    
class Chromosome(BaseChromosome):#(NamedTuple):
    
    def __init__(self,
                 a_filter_size:     Tuple[int],
                 a_include_BN:      bool,
                 a_output_channels: int,
                 activation_type:   tf.keras.layers.Layer,
                 b_filter_size:     Tuple[int],
                 b_include_BN:      bool,
                 b_output_channels: int,
                 include_pool:      bool, 
                 pool_type:         tf.keras.layers.Layer,
                 include_skip:      bool,
                 **kwargs):
        
        super().__init__()
        
        self.__chromosome = {k:v for k,v in locals().items() if k not in ['self', 'kwargs'] and not k.startswith('__')}
#         self.__chromosomes = {k:v for k,v in locals().items() if k!='self' and not k.startswith('__')}

    @property
    def chromosome(self):
        return self.__chromosome

    def __repr__(self):
        return '\n'.join([f'{k}:\n\t{v}' for k,v in self.chromosome.items()])

    
        
#     def __validate_args(self, *args, **kwargs):
#         validated = {}
#         for k,v in kwargs().items():
#             if k!='self' and not k.startswith('_'):
#                 validated[k] = v
#         return validated

In [48]:
# class ChromosomeOptions(NamedTuple):
#     """
#     Container class for encapsulating variable-length lists of potential gene variants (individual hyperparameters).
#     To be used as a reservoir from which to sample a complete chromosome made up of 1 variant per gene.

#     Gene: The unique identifier of a particular hyperparameter that may reference any of a set of possible variant values.
#     Variant: The particular value of a gene. Used to refer to the 1 value for a single chromosome instance, or 1 value from a set of gene options.

#     Args:
#         NamedTuple ([type]): [description]
#     """
#     a_filter_size:     List[Tuple[int]]            = [(1,1), (3,3), (5,5), (7,7), (9,9)]
#     a_include_BN:      List[bool]                  = [True, False]
#     a_output_channels: List[int]                   = [8, 16, 32, 64, 128, 256, 512]
#     activation_type:   List[tf.keras.layers.Layer] = [ReLU, ELU, LeakyReLU]
#     b_filter_size:     List[Tuple[int]]            = [(1,1), (3,3), (5,5), (7,7), (9,9)]
#     b_include_BN:      List[bool]                  = [True, False]
#     b_output_channels: List[int]                   = [8, 16, 32, 64, 128, 256, 512]
#     include_pool:      List[bool]                  = [True, False]
#     pool_type:         List[tf.keras.layers.Layer] = [MaxPool2D, AveragePooling2D]
#     include_skip:      List[bool]                  = [True, False]
        
        
#     def generate_chromosome(self, seed=None):
        

# from prodict import Prodict
    
    
# # class ChromosomeOptions(Prodict):
# # class ChromosomeOptions(Dict):
# class ChromosomeOptions(NamedTuple):
#     """
#     Container class for encapsulating variable-length lists of potential gene variants (individual hyperparameters).
#     To be used as a reservoir from which to sample a complete chromosome made up of 1 variant per gene.

#     Gene: The unique identifier of a particular hyperparameter that may reference any of a set of possible variant values.
#     Variant: The particular value of a gene. Used to refer to the 1 value for a single chromosome instance, or 1 value from a set of gene options.

#     Args:
#         NamedTuple ([type]): [description]
#     """
#     a_filter_size:     List[Tuple[int]]            = [(1,1), (3,3), (5,5), (7,7), (9,9)]
#     a_include_BN:      List[bool]                  = [True, False]
#     a_output_channels: List[int]                   = [8, 16, 32, 64, 128, 256, 512]
#     activation_type:   List[tf.keras.layers.Layer] = [ReLU, ELU, LeakyReLU]
#     b_filter_size:     List[Tuple[int]]            = [(1,1), (3,3), (5,5), (7,7), (9,9)]
#     b_include_BN:      List[bool]                  = [True, False]
#     b_output_channels: List[int]                   = [8, 16, 32, 64, 128, 256, 512]
#     include_pool:      List[bool]                  = [True, False]
#     pool_type:         List[tf.keras.layers.Layer] = [MaxPool2D, AveragePooling2D]
#     include_skip:      List[bool]                  = [True, False]
        
#     def keys(self):
#         return self._asdict().keys()
    
#     def values(self):
#         return self._asdict().values()
    
#     def items(self):
#         return self._asdict().items()
        
# #     def generate_chromosome(self, seed=None):
        

        
        

# class ChromosomeOptions:

#     a_filter_size:     List[Tuple[int]]            = [(1,1), (3,3), (5,5), (7,7), (9,9)]
#     a_include_BN:      List[bool]                  = [True, False]
#     a_output_channels: List[int]                   = [8, 16, 32, 64, 128, 256, 512]
    
#     def __init__(self, *args, **kwargs):
#         self = Bunch(**kwargs)

# options = ChromosomeOptions()

# dir((options))

# # options.__dict__

# 'a_output_channels' in options

# options.__class__.__dict__.values()

In [63]:
ActivationLayers = Box(ReLU=ReLU, ELU=ELU, LeakyReLU=LeakyReLU)
PoolingLayers = Box(MaxPool2D=MaxPool2D, AveragePooling2D=AveragePooling2D)

class ChromosomeOptions(BaseOptions): #object):
# class ChromosomeOptions(Prodict):
# class ChromosomeOptions(Dict):
# class ChromosomeOptions(Box):
    """
    Container class for encapsulating variable-length lists of potential gene variants (individual hyperparameters).
    To be used as a reservoir from which to sample a complete chromosome made up of 1 variant per gene.
    
    This should be logged for describing the scope of a given AutoML experiment's hyperparameter search space

    Gene: The unique identifier of a particular hyperparameter that may reference any of a set of possible variant values.
    Variant: The particular value of a gene. Used to refer to the 1 value for a single chromosome instance, or 1 value from a set of gene options.

    Args:
        NamedTuple ([type]): [description]
    """

    def __init__(self,
                 include_layer:     Optional[bool]              = [True, False]
                 a_filter_size:     List[Tuple[int]]            = [(1,1), (3,3), (5,5), (7,7), (9,9)],
                 a_include_BN:      List[bool]                  = [True, False],
                 a_output_channels: List[int]                   = [8, 16, 32, 64, 128, 256, 512],
                 activation_type:   List[str]                   = ['ReLU', 'ELU', 'LeakyReLU'],
                 b_filter_size:     List[Tuple[int]]            = [(1,1), (3,3), (5,5), (7,7), (9,9)],
                 b_include_BN:      List[bool]                  = [True, False],
                 b_output_channels: List[int]                   = [8, 16, 32, 64, 128, 256, 512],
                 include_pool:      List[bool]                  = [True, False],
                 pool_type:         List[str]                   = ['MaxPool2D', 'AveragePooling2D'],
                 include_skip:      List[bool]                  = [True, False],
                 **kwargs):
#         super().__init__()|
#                  activation_type:   List[tf.keras.layers.Layer] = [ReLU, ELU, LeakyReLU],
# pool_type:         List[tf.keras.layers.Layer] = [MaxPool2D, AveragePooling2D],
        
#         self.__chromosomes = {k:v for k,v in locals().items() if k not in ['self']}
#         self.__chromosomes = self.__validate_args(**{k:v for k,v in locals().items() if k!='self'})
        self.__chromosomes = {k:v for k,v in locals().items() if k not in ['self', 'kwargs'] and not k.startswith('__')}
        print(self.__chromosomes)
#         print(self.__chromosomes['validated'])#.keys())
        
        self.__activation_type = [ActivationLayers[act_layer] for act_layer in self.__chromosomes['activation_type']]
        self.__pool_type = [PoolingLayers[pool_layer] for pool_layer in self.__chromosomes['pool_type']]
        
        if 'seed' in kwargs:
            self.rng = np.random.default_rng(seed)
        else:
            self.rng = np.random.default_rng()
        
        self.__indent = None

    def __dumps__(self, indent=None):
        return json.dumps(self.chromosomes, indent=indent)
    
    @classmethod
    def __loads__(cls, json_string):
        loaded = cls()
        loaded.__init__(**json.loads(json_string))
        return loaded
        
        
    def __validate_args(self, *args, **kwargs):
        validated = {}
        for k,v in kwargs.items():
            if k!='self' and not k.startswith('_'):
                validated[k] = v
        return validated
    
    
    def sample_k_variants_from_gene(self, gene: str, k: int=1):
        '''
        Randomly sample the list of variants corresponding to the key indicated by the first arg, 'gene'. Produce a random sequence of length k, with the default==1.
        
        Note: If k==1: this automatically returns a single unit from the variants list, which may or may not be a scalar object (e.g. int, str, float)
        If k > 1: then the sampled variants will always be returned in a list.
        
        '''
        all_variants = self.chromosomes[gene]
        variant_idx = self.rng.integers(low=0, high=len(all_variants), size=k)
        sampled_variants = [all_variants[idx] for idx in variant_idx.tolist()]
        if k==1:
            sampled_variants = sampled_variants[0]
        return sampled_variants
    
    def generate_chromosome(self, phase: int, seed=None):
        '''
        Primary function for utilizing a ChromosomeOptions object during experimentation.
        Running this function will randomly generate a new Chromosome instance for which each genetic variant is randomly sampled from this object's contained data,
        in the form of mappings between gene names as keys, and lists of variants as values.
        '''
        return Chromosome(**{gene:self.sample_k_variants_from_gene(gene) for gene in self.chromosomes.keys()})
    
    def generate_k_chromosomes(self, k: int=1, seed=None):
        return [self.generate_chromosome(seed=seed) for _ in range(k)]
        

    @property
    def chromosomes(self):
        return self.__chromosomes
        
    if phase == 0:
        return {
        'a_filter_size': options_phase0['a_filter_size'][np.random.randint(len(options_phase0['a_filter_size']))],
        'a_include_BN': options_phase0['a_include_BN'][np.random.randint(len(options_phase0['a_include_BN']))],
        'a_output_channels': options_phase0['a_output_channels'][np.random.randint(len(options_phase0['a_output_channels']))],
        'activation_type': options_phase0['activation_type'][np.random.randint(len(options_phase0['activation_type']))],
        'b_filter_size': options_phase0['b_filter_size'][np.random.randint(len(options_phase0['b_filter_size']))],
        'b_include_BN': options_phase0['b_include_BN'][np.random.randint(len(options_phase0['b_include_BN']))],
        'b_output_channels': options_phase0['b_output_channels'][np.random.randint(len(options_phase0['b_output_channels']))],
        'include_pool': options_phase0['include_pool'][np.random.randint(len(options_phase0['include_pool']))],
        'pool_type': options_phase0['pool_type'][np.random.randint(len(options_phase0['pool_type']))],
        'include_skip': options_phase0['include_skip'][np.random.randint(len(options_phase0['include_skip']))]
        }
    else:
        return {
        'a_filter_size': options['a_filter_size'][np.random.randint(len(options['a_filter_size']))],
        'a_include_BN': options['a_include_BN'][np.random.randint(len(options['a_include_BN']))],
        'a_output_channels': options['a_output_channels'][np.random.randint(len(options['a_output_channels']))],
        'b_filter_size': options['b_filter_size'][np.random.randint(len(options['b_filter_size']))],
        'b_include_BN': options['b_include_BN'][np.random.randint(len(options['b_include_BN']))],
        'b_output_channels': options['b_output_channels'][np.random.randint(len(options['b_output_channels']))],
        'include_pool': options['include_pool'][np.random.randint(len(options['include_pool']))],
        'pool_type': options['pool_type'][np.random.randint(len(options['pool_type']))],
        'include_layer': options['include_layer'][np.random.randint(len(options['include_layer']))],
        'include_skip': options['include_skip'][np.random.randint(len(options['include_skip']))]
        }
options = ChromosomeOptions()
new_options = options.generate_chromosome()
opt_str = options.serialized
loaded = ChromosomeOptions.from_serialized(opt_str)
options==loaded
new_options







class Trial(stateful.Stateful):

    def __init__(self,
                 hyperparameters,
                 trial_id=None,
                 status=TrialStatus.RUNNING):
        self.hyperparameters = hyperparameters
        self.trial_id = generate_trial_id() if trial_id is None else trial_id

        self.metrics = metrics_tracking.MetricsTracker()
        self.score = None
        self.best_step = None
        self.status = status

    def summary(self):
        """Displays a summary of this Trial."""
        print('Trial summary')

        print('Hyperparameters:')
        self.display_hyperparameters()

        if self.score is not None:
            print('Score: {}'.format(self.score))

    def display_hyperparameters(self):
        if self.hyperparameters.values:
            for hp, value in self.hyperparameters.values.items():
                print(hp + ':', value)
        else:
            print('default configuration')

    def get_state(self):
        return {
            'trial_id': self.trial_id,
            'hyperparameters': self.hyperparameters.get_config(),
            'metrics': self.metrics.get_config(),
            'score': self.score,
            'best_step': self.best_step,
            'status': self.status
        }

    def set_state(self, state):
        self.trial_id = state['trial_id']
        hp = hp_module.HyperParameters.from_config(
            state['hyperparameters']
        )
        self.hyperparameters = hp
        self.metrics = metrics_tracking.MetricsTracker.from_config(state['metrics'])
        self.score = state['score']
        self.best_step = state['best_step']
        self.status = state['status']

    @classmethod
    def from_state(cls, state):
        trial = cls(hyperparameters=None)
        trial.set_state(state)
        return trial

    @classmethod
    def load(cls, fname):
        with tf.io.gfile.GFile(fname, 'r') as f:
            state_data = f.read()
        return cls.from_state(state_data)

{'a_filter_size': [(1, 1), (3, 3), (5, 5), (7, 7), (9, 9)], 'a_include_BN': [True, False], 'a_output_channels': [8, 16, 32, 64, 128, 256, 512], 'activation_type': ['ReLU', 'ELU', 'LeakyReLU'], 'b_filter_size': [(1, 1), (3, 3), (5, 5), (7, 7), (9, 9)], 'b_include_BN': [True, False], 'b_output_channels': [8, 16, 32, 64, 128, 256, 512], 'include_pool': [True, False], 'pool_type': ['MaxPool2D', 'AveragePooling2D'], 'include_skip': [True, False]}
{'a_filter_size': [(1, 1), (3, 3), (5, 5), (7, 7), (9, 9)], 'a_include_BN': [True, False], 'a_output_channels': [8, 16, 32, 64, 128, 256, 512], 'activation_type': ['ReLU', 'ELU', 'LeakyReLU'], 'b_filter_size': [(1, 1), (3, 3), (5, 5), (7, 7), (9, 9)], 'b_include_BN': [True, False], 'b_output_channels': [8, 16, 32, 64, 128, 256, 512], 'include_pool': [True, False], 'pool_type': ['MaxPool2D', 'AveragePooling2D'], 'include_skip': [True, False]}
{'a_filter_size': [[1, 1], [3, 3], [5, 5], [7, 7], [9, 9]], 'a_include_BN': [True, False], 'a_output_channel

a_filter_size:
	(5, 5)
a_include_BN:
	True
a_output_channels:
	512
activation_type:
	ReLU
b_filter_size:
	(3, 3)
b_include_BN:
	False
b_output_channels:
	128
include_pool:
	True
pool_type:
	MaxPool2D
include_skip:
	False

In [ ]:
config

## 1. INTERESTING REFACTOR IDEA:
    TODO: Refactor chromosome structure to standardize the configuration options for repeated model structures
    ### (3 AM 11/27/20)

    e.g. Create a separate ConvOptions(NamedTuple) class to contain all 3 options:
        filter_size
    include_BN
    output_channels

    Then in each "ChromosomeOptions" (consider making each of those a chromosome, and upgrading what's now a chromosome to a full Genome)
    store a separate ConvOptions for layer a and layer b, separately.


## 2. TODO: 
    Consider transferring mutate() method from Organism to Chromosome, while potentially keeping crossover() method as part of organism's namespace. Purpose is to encapsulate functionality as close as possible with the data/abstractions it will operate on


## 3. To Consider:
    How can I quantify the information coverage and computational complexity of a given set of chromosome options? 

        a. Start with the raw # of permutations of all chromosome options
        b. Adjust by the expected coverage for each variant. E.g. How much of the hyperparameter space are we covering in our naive uniform grid search?

In [ ]:
options_phase0 = {
    'a_filter_size': [(1,1), (3,3), (5,5), (7,7), (9,9)],
    'a_include_BN': [True, False],
    'a_output_channels': [8, 16, 32, 64, 128, 256, 512],
    'activation_type': [ReLU, ELU, LeakyReLU],
    'b_filter_size': [(1,1), (3,3), (5,5), (7,7), (9,9)],
    'b_include_BN': [True, False],
    'b_output_channels': [8, 16, 32, 64, 128, 256, 512],
    'include_pool': [True, False],
    'pool_type': [MaxPool2D, AveragePooling2D],
    'include_skip': [True, False]
}

options = {
    'include_layer': [True, False],
    'a_filter_size': [(1,1), (3,3), (5,5), (7,7), (9,9)],
    'a_include_BN': [True, False],
    'a_output_channels': [8, 16, 32, 64, 128, 256, 512],
    'b_filter_size': [(1,1), (3,3), (5,5), (7,7), (9,9)],
    'b_include_BN': [True, False],
    'b_output_channels': [8, 16, 32, 64, 128, 256, 512],
    'include_pool': [True, False],
    'pool_type': [MaxPool2D, AveragePooling2D],
    'include_skip': [True, False]
}


def random_hyper(phase):
    if phase == 0:
        return {
        'a_filter_size': options_phase0['a_filter_size'][np.random.randint(len(options_phase0['a_filter_size']))],
        'a_include_BN': options_phase0['a_include_BN'][np.random.randint(len(options_phase0['a_include_BN']))],
        'a_output_channels': options_phase0['a_output_channels'][np.random.randint(len(options_phase0['a_output_channels']))],
        'activation_type': options_phase0['activation_type'][np.random.randint(len(options_phase0['activation_type']))],
        'b_filter_size': options_phase0['b_filter_size'][np.random.randint(len(options_phase0['b_filter_size']))],
        'b_include_BN': options_phase0['b_include_BN'][np.random.randint(len(options_phase0['b_include_BN']))],
        'b_output_channels': options_phase0['b_output_channels'][np.random.randint(len(options_phase0['b_output_channels']))],
        'include_pool': options_phase0['include_pool'][np.random.randint(len(options_phase0['include_pool']))],
        'pool_type': options_phase0['pool_type'][np.random.randint(len(options_phase0['pool_type']))],
        'include_skip': options_phase0['include_skip'][np.random.randint(len(options_phase0['include_skip']))]
        }
    else:
        return {
        'a_filter_size': options['a_filter_size'][np.random.randint(len(options['a_filter_size']))],
        'a_include_BN': options['a_include_BN'][np.random.randint(len(options['a_include_BN']))],
        'a_output_channels': options['a_output_channels'][np.random.randint(len(options['a_output_channels']))],
        'b_filter_size': options['b_filter_size'][np.random.randint(len(options['b_filter_size']))],
        'b_include_BN': options['b_include_BN'][np.random.randint(len(options['b_include_BN']))],
        'b_output_channels': options['b_output_channels'][np.random.randint(len(options['b_output_channels']))],
        'include_pool': options['include_pool'][np.random.randint(len(options['include_pool']))],
        'pool_type': options['pool_type'][np.random.randint(len(options['pool_type']))],
        'include_layer': options['include_layer'][np.random.randint(len(options['include_layer']))],
        'include_skip': options['include_skip'][np.random.randint(len(options['include_skip']))]
        }

In [ ]:
class Organism:
    def __init__(self,
                 data: Dict[str,tf.data.Dataset],
                 config=None,
                 chromosome={},
                 phase=0,
                 prevBestOrganism=None):
        '''
        config is a . accessible dict object containing model params that will stay constant during evolution
        chromosome is a dictionary of genes
        phase is the phase that the individual belongs to
        prevBestOrganism is the best organism of the previous phase
        
        TODO:
        
        1. implement to_json and from_json methods for copies
        2. Separate out step where organism is associated with a dataset
        '''
        self.data = data
        self.train_data = data['train']
        self.val_data = data['val']
        self.test_data = data['test']
        self.config = config
        self.phase = phase
        self.chromosome = chromosome
        self.prevBestOrganism=prevBestOrganism
        if phase != 0:
            # In a later stage, the model is made by
            # attaching new layers to the prev best model
            self.last_model = prevBestOrganism.model
    
    @property
    def config(self):
        return self._config
    
    @config.setter
    def config(self, config=None):
        config = config or OmegaConf.create({})
        config.input_shape = config.input_shape or (224,224,3)
        config.output_size = config.output_size or 38
        config.epochs_per_organism = config.epochs_per_organism or 5
        self._config = config
    
    def build_model(self):
        '''
        This is the function to build the keras model
        '''
        K.clear_session()
        gc.collect()
        inputs = Input(shape=self.config.input_shape)
        if self.phase != 0:
            # Slice the prev best model # Use the model as a layer # Attach new layer to the sliced model
            intermediate_model = Model(inputs=self.last_model.input,
                                       outputs=self.last_model.layers[-3].output)
            for layer in intermediate_model.layers:
                # To make the iteration efficient
                layer.trainable = False
            inter_inputs = intermediate_model(inputs)
            x = Conv2D(filters=self.chromosome['a_output_channels'],
                       padding='same',
                       kernel_size=self.chromosome['a_filter_size'],
                       use_bias=self.chromosome['a_include_BN'])(inter_inputs)
            # This is to ensure that we do not randomly chose anothere activation
            self.chromosome['activation_type'] = self.prevBestOrganism.chromosome['activation_type']
        else:
            # For PHASE 0 only
            # input layer
            x = Conv2D(filters=self.chromosome['a_output_channels'],
                       padding='same',
                       kernel_size=self.chromosome['a_filter_size'],
                       use_bias=self.chromosome['a_include_BN'])(inputs)
            
        if self.chromosome['a_include_BN']:
            x = BatchNormalization()(x)
        x = self.chromosome['activation_type']()(x)
        if self.chromosome['include_pool']:
            x = self.chromosome['pool_type'](strides=(1,1),
                                             padding='same')(x)
        if self.phase != 0 and self.chromosome['include_layer'] == False:
            # Except for PHASE0, there is a choice for
            # the number of layers that the model wants
            if self.chromosome['include_skip']:
                y = Conv2D(filters=self.chromosome['a_output_channels'],
                           kernel_size=(1,1),
                           padding='same')(inter_inputs)
                x = Add()([y,x])
            x = GlobalAveragePooling2D()(x)
            x = Dense(self.output_shape, activation='softmax')(x)
        else:
            # PHASE0 or no skip
            # in the tail
            x = Conv2D(filters=self.chromosome['b_output_channels'],
                       padding='same',
                       kernel_size=self.chromosome['b_filter_size'],
                       use_bias=self.chromosome['b_include_BN'])(x)
            if self.chromosome['b_include_BN']:
                x = BatchNormalization()(x)
            x = self.chromosome['activation_type']()(x)
            if self.chromosome['include_skip']:
                y = Conv2D(filters=self.chromosome['b_output_channels'],
                           padding='same',
                           kernel_size=(1,1))(inputs)
                x = Add()([y,x])
            x = GlobalAveragePooling2D()(x)
            x = Dense(self.config.output_size, activation='softmax')(x)
        self.model = Model(inputs=[inputs], outputs=[x])
        self.model.compile(optimizer='adam',
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])
        
    def fitnessFunction(self,
                        train_data,
                        val_data,
                        generation_number):
        '''
        This function is used to calculate the
        fitness of an individual.
        '''
        wandb.init(**get_wandb_credentials(phase=self.phase,
                                           generation_number=generation_number))
        
        self.model.fit(train_data,
                       epochs=self.config.epochs_per_organism,
                       callbacks=[WandbCallback()],
                       verbose=1)
        _, self.fitness = self.model.evaluate(val_data,
                                              verbose=1)
    def crossover(self,
                  partner,
                  generation_number):
        '''
        This function helps in making children from two
        parent individuals.
        '''
        child_chromosome = {}
        endpoint = np.random.randint(low=0, high=len(self.chromosome))
        for idx, key in enumerate(self.chromosome):
            if idx <= endpoint:
                child_chromosome[key] = self.chromosome[key]
            else:
                child_chromosome[key] = partner.chromosome[key]
        child = Organism(chromosome=child_chromosome,
                         data=self.data,
                         config=self.config,
                         phase=self.phase,
                         prevBestOrganism=self.prevBestOrganism)
        child.build_model()
        child.fitnessFunction(self.train_data,
                              self.val_data,
                              generation_number=generation_number)
        return child
    
    def mutation(self, generation_number):
        '''
        One of the gene is to be mutated.
        '''
        index = np.random.randint(0, len(self.chromosome))
        key = list(self.chromosome.keys())[index]
        if  self.phase != 0:
            self.chromosome[key] = options[key][np.random.randint(len(options[key]))]
        else:
            self.chromosome[key] = options_phase0[key][np.random.randint(len(options_phase0[key]))]
        self.build_model()
        self.fitnessFunction(self.train_data,
                             self.val_data,
                             generation_number=generation_number)
    
    def show(self):
        '''
        Util function to show the individual's properties.
        '''
        pp.pprint(self.config)
        pp.pprint(self.chromosome)
        
    
def get_wandb_credentials(phase: int, generation_number: int):
    return dict(entity="jrose",
                project=f"vlga-plant_village",
                group='KAGp{}'.format(phase),
                job_type='g{}'.format(generation_number))


def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

# Generation
This is a class that hold generations of models.

1. fitSurvivalRate - The amount of fit individuals we want in the next generation.
2. unfitSurvivalProb - The probability of sending unfit individuals
3. mutationRate - The mutation rate to change genes in an individual.
4. phase - The phase that the generation belongs to.
5. population_size - The amount of individuals that the generation consists of.
6. prevBestOrganism - The best organism (individual) is the last phase

In [ ]:
class Generation:
    def __init__(self,
                 data,
                 generation_config,
                 organism_config,
                 phase,
                 prevBestOrganism):
        self.data = data
        self.config = generation_config
        self.organism_config = organism_config
        self.population = []
        self.generation_number = 0
        self.phase = phase
        # creating the first population: GENERATION_0
        # can be thought of as the setup function
        self.prevBestOrganism = prevBestOrganism or None
        self.initialize_population()
        
    @property
    def config(self):
        return self._config
    
    @config.setter
    def config(self, config=None):
        config = config or OmegaConf.create({})
        config.population_size = config.population_size or 5
        config.num_generations_per_phase = config.num_generations_per_phase or 3
        config.fitSurvivalRate = config.fitSurvivalRate or 0.5
        config.unfitSurvivalProb = config.unfitSurvivalProb or 0.2
        config.mutationRate = config.mutationRate or 0.1
        config.num_phases = config.num_phases or 5
        
        self._config = config
        self.__dict__.update(config)
        
        
    def initialize_population(self):
        '''
        1. Create self.population_size individual organisms from scratch by randomly sampling an initial set of hyperparameters (a chromosome)
        2. As each is instantiated, build its model
        3. Assess their fitness one-by-one
        4. Sort models by relative fitness so we have a (potentially) new Best Organism (best model)
        4. Increment generation number to 1
        '''

        for idx in range(self.population_size):
            print(f'Creating, training then testing organism {idx} of generation {self.generation_number} and phase {self.phase}')
            org = Organism(chromosome=random_hyper(self.phase),
                           data=self.data,
                           config=self.organism_config,
                           phase=self.phase,
                           prevBestOrganism=self.prevBestOrganism)
            org.build_model()
            org.fitnessFunction(org.data['train'],
                                org.data['test'],
                                generation_number=self.generation_number)
            self.population.append(org)

        # sorts the population according to fitness (high to low)
        self.sortModel()
        self.generation_number += 1

    def sortModel(self):
        '''
        sort the models according to the 
        fitness in descending order.
        '''
        fitness = [ind.fitness for ind in self.population]
        sort_index = np.argsort(fitness)[::-1]
        self.population = [self.population[index] for index in sort_index]

    def generate(self):
        '''
        Generate a new generation in the same phase
        '''
        number_of_fit = int(self.population_size * self.fitSurvivalRate)
        new_pop = self.population[:number_of_fit]
        for individual in self.population[number_of_fit:]:
            if np.random.rand() <= self.unfitSurvivalProb:
                new_pop.append(individual)
        for index, individual in enumerate(new_pop):
            if np.random.rand() <= self.mutationRate:
                new_pop[index].mutation(generation_number=self.generation_number)
        fitness = [ind.fitness for ind in new_pop]
        children=[]
        for idx in range(self.population_size-len(new_pop)):
            parents = np.random.choice(new_pop, replace=False, size=(2,), p=softmax(fitness))
            A=parents[0]
            B=parents[1]
            child=A.crossover(B, generation_number=self.generation_number)
            children.append(child)
        self.population = new_pop+children
        self.sortModel()
        self.generation_number+=1

    def evaluate(self, last=False):
        '''
        Evaluate the generation
        '''
        fitness = [ind.fitness for ind in self.population]
        
        wandb.log({'population_size':len(fitness)}, commit=False)
        wandb.log({'Best fitness': fitness[0]}, commit=False)
        wandb.log({'Average fitness': sum(fitness)/len(fitness)})
        
        self.population[0].show()
        if last:
            BestOrganism = self.population[0]
            model_path = f'best-model-phase_{self.phase}.png'
            tf.keras.utils.plot_model(BestOrganism.model, to_file=model_path)
            wandb.log({"best_model": [wandb.Image(model_path, caption=f"Best Model phase_{self.phase}")]})
            log_high_loss_examples(BestOrganism.test_dataset,
                                   BestOrganism.model, 
                                   k=32)
            
            return BestOrganism
#             return self.population[0]

In [ ]:
# population_size = 5
# num_generations_per_phase = 3
# fitSurvivalRate = 0.5
# unfitSurvivalProb = 0.2
# mutationRate = 0.1
# num_phases = 5
# prevBestOrganism = None
prevBestOrganism = None

for phase in range(generation_config.num_phases):
    print("PHASE {}".format(phase))
    generation = Generation(data=data,
                            generation_config=generation_config,
                            organism_config=organism_config,
                            phase=phase,
                            prevBestOrganism=prevBestOrganism)
#     while generation.generation_number < num_generations_per_phase:
    generation.generate()
    if generation.generation_number == generation.num_generations_per_phase:
        # Last generation is the phase
        # print('I AM THE BEST IN THE PHASE')
        prevBestOrganism = generation.evaluate(last=True)
#         model_path = f'best-model-phase_{phase}.png'
#         tf.keras.utils.plot_model(prevBestOrganism.model, to_file=model_path)
#         wandb.log({"best_model": [wandb.Image(model_path, caption=f"Best Model phase_{phase}")]})
#         log_high_loss_examples(prevBestOrganism.test_dataset,
#                                prevBestOrganism.model, 
#                                k=32)
    else:
        generation.evaluate()

In [ ]:
%debug

In [ ]:
### 1. Using tfds.features.ClassLabel

# feature_labels = tfds.features.ClassLabel(names=vocab)
# data = ['Potato___healthy',
#         'Potato___Late_blight',
#         'Raspberry___healthy',
#         'Soybean___healthy',
#         'Squash___Powdery_mildew',
#         'Strawberry___healthy',
#         'Strawberry___Leaf_scorch',
#         'Tomato___Bacterial_spot',
#         'Tomato___Early_blight',
#         'Tomato___healthy']

# data += data[::-1]
# print([feature_labels.str2int(label) for label in data])
# data = train_data
# data_enc = data.map(lambda x,y: (x, feature_labels.int2str(y)))

### 2. Using StringLookup and CategoryEncoding Layers

# layer = StringLookup(vocabulary=vocab, num_oov_indices=0, mask_token=None)
# i_layer = StringLookup(vocabulary=layer.get_vocabulary(), invert=True)
# int_data = layer(data)

# print(len(layer.get_vocabulary()))
# print(len(class_encoder.class_list))
# print(set(layer.get_vocabulary())==set(class_encoder.class_list))

# i_layer = StringLookup(vocabulary=layer.get_vocabulary(), invert=True)
# int_data = layer(data)

# print(layer(data))
# print(i_layer(int_data))

In [ ]:
# # from tensorflow.keras.layers.experimental.preprocessing import StringLookup, CategoryEncoding
# # data = tf.constant(["a", "b", "c", "b", "c", "a"])
# # # Use StringLookup to build an index of the feature values
# # indexer = StringLookup()
# # indexer.adapt(data)
# # # Use CategoryEncoding to encode the integer indices to a one-hot vector
# # encoder = CategoryEncoding(output_mode="binary")
# # encoder.adapt(indexer(data))
# # # Convert new test data (which includes unknown feature values)
# # test_data = tf.constant(["a", "b", "c", "d", "e", ""])
# # encoded_data = encoder(indexer(test_data))
# # print(encoded_data)

# vocab = ["a", "b", "c", "d"]
# data = tf.constant([["a", "c", "d"], ["d", "z", "b"]])
# layer = StringLookup(vocabulary=vocab)
# i_layer = StringLookup(vocabulary=layer.get_vocabulary(), invert=True)
# int_data = layer(data)

# print(layer(data))
# print(i_layer(int_data))